In [1]:
# 토치 식구들
import torch
import torch.nn as nn
import torch.nn.functional as F


# 넘파이는 가족이다
import numpy as np

# 각종 그래프 찍어볼 때 사용할 것이다.
import matplotlib.pyplot as plt

# GPU 쓸 수 있으면 쓰자
device = torch.device('cuda' if torch.cuda.is_available()==True else 'cpu')


In [2]:
# 데이터 불러오기
train = np.loadtxt("mnist_train.csv", delimiter=',', dtype=float)
test = np.loadtxt("mnist_test.csv", delimiter=',', dtype=float)

train_X = torch.FloatTensor(train[:, 1:])
train_Y = torch.FloatTensor(train[:, :1])
test_X = torch.FloatTensor(test[:, 1:])
test_Y = torch.FloatTensor(test[:, :1])

print(train_X.shape)
print(train_Y.shape)

print(test_X.shape)
print(test_Y.shape)

torch.Size([60000, 784])
torch.Size([60000, 1])
torch.Size([10000, 784])
torch.Size([10000, 1])


In [3]:
# Y를 One-hot으로 바꿔준다
Y_one_hot_ = torch.squeeze(F.one_hot(train_Y.to(torch.int64), num_classes=10))
Y_one_hot_test = torch.squeeze(F.one_hot(test_Y.to(torch.int64), num_classes=10))

In [4]:
# MNIST 분류모델 구성
model = nn.Sequential(
    nn.Linear(28*28, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 10),
    nn.Softmax()
)


In [5]:
# Loss, Optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

In [6]:
n_ephocs = 500
batch_size = 3000

train_X = train_X/255

for i in range(n_ephocs):
  for j in range(int(len(train_X)/batch_size)):
     outputs = model(train_X[(j*batch_size):((j+1)*batch_size), :])
     loss = loss_fn(outputs, Y_one_hot_[(j*batch_size):((j+1)*batch_size), :].to(torch.float32))
     #print(outputs[:5], torch.squeeze(Y_one_hot_[(j*batch_size):((j+1)*batch_size), :]).to(torch.float32)[:5])

     optimizer.zero_grad()
     loss.backward()
     optimizer.step()
  if (i+1)%50 == 0:
    print("epoch{0} LOSS：{1}".format(i+1, loss.item()))

C:\Users\danie\anaconda3\envs\csh\Lib\site-packages\torch\nn\modules\container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch50 LOSS：1.5767327547073364
epoch100 LOSS：1.5663315057754517
epoch150 LOSS：1.4805481433868408
epoch200 LOSS：1.4689695835113525
epoch250 LOSS：1.4682466983795166
epoch300 LOSS：1.4661779403686523
epoch350 LOSS：1.4661260843276978
epoch400 LOSS：1.4658095836639404
epoch450 LOSS：1.4658191204071045
epoch500 LOSS：1.4663362503051758


In [7]:
# 모델로 5개만 찍어보기
model.eval()
test_X = test_X/255
predict = model(test_X)
predict = torch.argmax(predict, dim=1)
print(predict[:5].to(torch.int32))
print(torch.squeeze(test_Y[:5, :]).to(torch.int32))

tensor([7, 2, 1, 0, 4], dtype=torch.int32)
tensor([7, 2, 1, 0, 4], dtype=torch.int32)


In [9]:
correct = 0
for i in range(len(test_X)):
  if predict[i].to(torch.int32) == torch.squeeze(test_Y[:, :]).to(torch.int32)[i]:
    correct += 1

print("ACC: {0}%".format(correct/len(test_X)))

ACC: 0.9703%
